In [ ]:
import torch
import torch.nn as nn
import math
import numpy as np

from UNet import UNet

In [ ]:
img = torch.randn((10, 1, 32, 32))
model = UNet()
img = model (img, t)

In [ ]:
class DiffusionModel(nn.Module):
    
    def __init__(self, T, model, device):

        self.T = T
        self.function_aproximator = model.to(device)
        self.device = device
        
        self.beta = torch.linspace(1e-4, 0.02, T).to(device)
        self.alpha = 1. - self.beta
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)

    def training(self, batch_size, optimizer):
        """
        Algorithm 1 in Denoising Diffusion Probabilistic Model
        """
        
        x0 = sample_batch(batch_size, self.device)
        t = torch.randint(1, self.T+1, (batch_size,), device=self.device, dytpe=torch.long)
        eps = torch.randn_like(x0)

        # Take gradient descent step on
        alpha_bar_t = self.alpha_bar[t-1]
        eps_predicted = self.function_aproximator(
            torch.sqrt(alpha_bar_t) * x0 + torch.sqrt(1 - alpha_bar_t) * eps, t-1)
        
        loss = nn.functional.mse_loss(eps, eps_predicted)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        


    @torch.no_grad()
    def sampling(self):
        pass

In [ ]:
img.shape

In [ ]:
sum([p.numel() for p in model.parameters()]) / 1e6